In [ ]:
# import sagemaker as sage
# from time import gmtime, strftime

# sess = sage.Session()
# account = '452432741922'
# region = 'us-east-1'
# image = '452432741922.dkr.ecr.us-east-1.amazonaws.com/keras-test:latest'

In [ ]:
# model = sage.estimator.Estimator(image,
#                        role, 1, 'ml.p2.16xlarge',
#                        output_path="s3://{}/output".format(sess.default_bucket()),
#                        sagemaker_session=sess)

# model.fit(data_location)

In [1]:
import requests
import json
import cv2
import util
from config_reader import config_reader
import numpy as np
content_type = 'image/jpeg'
headers = {'content-type': content_type}
oriImg = cv2.imread('test.jpeg')
param, model_params = config_reader()
scale = [x * model_params['boxsize'] / oriImg.shape[0] for x in param['scale_search']][3]                      
heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 19))

imageToTest = cv2.resize(oriImg, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model_params['stride'], model_params['padValue'])     
_, img_encoded = cv2.imencode('.jpg', imageToTest_padded)

In [2]:
import boto3

client = boto3.client('sagemaker-runtime')

endpoint_name = "keras-test-2019-04-10-15-02-32-597"                                       # Your endpoint name.
content_type = "image/jpeg"                                        # The MIME type of the input data in the request body.
payload = img_encoded.tostring()                                             # Payload for inference.
 

In [3]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=payload
    )

print(response)       

{u'InvokedProductionVariant': 'AllTraffic', u'Body': <botocore.response.StreamingBody object at 0x7ff140f10f50>, u'ContentType': 'text/csv; charset=utf-8', 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '233f736f-991f-4496-8232-71dd286a894a', 'HTTPHeaders': {'x-amzn-requestid': '233f736f-991f-4496-8232-71dd286a894a', 'x-amzn-invoked-production-variant': 'AllTraffic', 'content-length': '55564', 'date': 'Wed, 10 Apr 2019 15:42:31 GMT', 'content-type': 'text/csv; charset=utf-8'}}}


In [4]:
resp = response['Body'].read()

In [5]:
from StringIO import StringIO
import pandas as pd

In [8]:
df = pd.read_csv(StringIO(resp),header=None)

In [18]:
to_parse = df.iloc[0,0]

In [20]:
to_parse = to_parse.replace('[[ ','')
to_parse = to_parse.replace('\n','')
to_parse = to_parse.replace(']]','')
to_parse.split(  )

['1.12943366e-04',
 '-1.99170954e-05',
 '7.93906365e-05',
 '...',
 '7.29355233e-05',
 '6.14081509e-05',
 '9.98405457e-01]',
 '[',
 '6.59970683e-05',
 '-2.86108661e-05',
 '2.10282924e-05',
 '...',
 '5.73969701e-05',
 '1.27583871e-05',
 '9.99846578e-01]',
 '[',
 '1.05659550e-04',
 '-2.00235172e-05',
 '8.34089078e-05',
 '...',
 '7.10179811e-05',
 '5.14465100e-05',
 '9.98590529e-01]',
 '...',
 '[',
 '1.15225281e-04',
 '5.57225867e-05',
 '3.87834647e-04',
 '...',
 '1.13739821e-04',
 '5.61066154e-05',
 '9.94679809e-01]',
 '[',
 '2.51533638e-04',
 '9.49491805e-05',
 '4.79963957e-04',
 '...',
 '2.62132759e-04',
 '1.48499428e-04',
 '9.92543817e-01]',
 '[',
 '5.13945357e-04',
 '2.39997913e-04',
 '6.52002171e-04',
 '...',
 '3.19770857e-04',
 '2.78469844e-04',
 '9.86387491e-01']

In [39]:
resp = resp.replace('\n','')
resp = resp.replace('"','')
resp = resp.replace(' ',',')
resp = resp.replace('[,',',')

In [40]:
resp

'[[,1.12943366e-04,-1.99170954e-05,,7.93906365e-05,...,,7.29355233e-05,,,6.14081509e-05,,9.98405457e-01],[,6.59970683e-05,-2.86108661e-05,,2.10282924e-05,...,,5.73969701e-05,,,1.27583871e-05,,9.99846578e-01],[,1.05659550e-04,-2.00235172e-05,,8.34089078e-05,...,,7.10179811e-05,,,5.14465100e-05,,9.98590529e-01],...,[,1.15225281e-04,,5.57225867e-05,,3.87834647e-04,...,,1.13739821e-04,,,5.61066154e-05,,9.94679809e-01],[,2.51533638e-04,,9.49491805e-05,,4.79963957e-04,...,,2.62132759e-04,,,1.48499428e-04,,9.92543817e-01],[,5.13945357e-04,,2.39997913e-04,,6.52002171e-04,...,,3.19770857e-04,,,2.78469844e-04,,9.86387491e-01]][[,8.1856633e-05,-2.5673906e-05,,4.0744464e-05,...,,6.2646257e-05,,,2.9193419e-05,,9.9935973e-01],[,7.3567149e-05,-2.7208998e-05,,3.0439207e-05,...,,5.9902559e-05,,,2.0603149e-05,,9.9961424e-01],[,7.3554118e-05,-2.7211410e-05,,3.0423013e-05,...,,5.9898244e-05,,,2.0589650e-05,,9.9961460e-01],...,[,7.0266753e-05,,3.7090271e-05,,2.9226032e-04,...,,8.9514491e-05,,,1.2487275e-05

In [36]:
import json

In [37]:
json.loads(resp)

ValueError: Expecting , delimiter: line 1 column 19 (char 18)